# Анализ базы данных у сервиса для чтения книг по подписке

### Загрузка библиотек. Подключение к базе данных:

In [13]:
import pandas as pd
from sqlalchemy import create_engine 

In [14]:
db_config = {'user': 'praktikum_student',
 'pwd': 'Sdf4$2;d-d30pp',
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, 
 'db': 'data-analyst-final-project-db'}
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

### Чтение данных

#### Данные о книгах:

In [15]:
books = "SELECT * FROM books LIMIT 10;"  

pd.io.sql.read_sql(books, con = engine) 

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
5,6,257,1st to Die (Women's Murder Club #1),424,2005-05-20,116
6,7,258,2nd Chance (Women's Murder Club #2),400,2005-05-20,116
7,8,260,4th of July (Women's Murder Club #4),448,2006-06-01,318
8,9,563,A Beautiful Mind,461,2002-02-04,104
9,10,445,A Bend in the Road,341,2005-04-01,116


#### Данные об авторах:

In [16]:
authors = "SELECT * FROM authors LIMIT 5;"  

pd.io.sql.read_sql(authors, con = engine) 

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


#### Данные об издательствах:

In [17]:
publishers = "SELECT * FROM publishers LIMIT 5;"  

pd.io.sql.read_sql(publishers, con = engine) 

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


#### Данные о пользовательских оценках книг:

In [18]:
ratings = "SELECT * FROM ratings LIMIT 5;"  

pd.io.sql.read_sql(ratings, con = engine) 

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


#### Данные о пользовательских обзорах на книги:

In [19]:
reviews  = "SELECT * FROM reviews LIMIT 5;"  

pd.io.sql.read_sql(reviews , con = engine) 

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


### Анализ данных

#### Количество книг, вышедших после 1 января 2000 года.

In [20]:
books_pub_after1Jan = "SELECT COUNT(*) AS cnt FROM books WHERE publication_date > '2000.01.01'"

pd.io.sql.read_sql(books_pub_after1Jan, con = engine) 

,cnt
0,819


<div style="border:solid gray 2px; padding: 20px">
    
Отфильтровав данные по дате, получили результат, что книг после 1 янв 2020 (не включая) было выпущено в количестве - 819.     

### Количество обзоров и средняя оценка для каждой книги.

Объединим таблицы books, rating, reviews

In [21]:
total_reviews_avg_rating="SELECT books.title AS title, COUNT(DISTINCT reviews.review_id) AS total_reviews, AVG(ratings.rating) AS avg_rating FROM books INNER JOIN reviews ON reviews.book_id = books.book_id INNER JOIN ratings ON ratings.book_id = books.book_id GROUP BY books.book_id ORDER BY books.book_id;"

pd.io.sql.read_sql(total_reviews_avg_rating, con = engine)

,title,total_reviews,avg_rating
0,'Salem's Lot,2,3.666667
1,1 000 Places to See Before You Die,1,2.500000
2,13 Little Blue Envelopes (Little Blue Envelope...,3,4.666667
3,1491: New Revelations of the Americas Before C...,2,4.500000
4,1776,4,4.000000
...,...,...,...
989,Wyrd Sisters (Discworld #6; Witches #2),3,3.666667
990,Xenocide (Ender's Saga #3),3,3.400000
991,Year of Wonders,4,3.200000
992,You Suck (A Love Story #2),2,4.500000


<div style="border:solid gray 2px; padding: 20px">

В представленной таблице содержится информация по всем книгам, количество обзоров и рейтинг по каждой. 

### Издательство, которое выпустило наибольшее число книг толще 50 страниц.

In [22]:
publishers= "SELECT publishers.publisher AS publisher,subquery.total_books  FROM publishers INNER JOIN(SELECT books.publisher_id AS publisher_id, COUNT(books.book_id) AS total_books FROM books WHERE num_pages > 50 GROUP BY books.publisher_id ) AS subquery ON subquery.publisher_id = publishers.publisher_id ORDER BY total_books DESC LIMIT 1;"

pd.io.sql.read_sql(publishers, con = engine)

,publisher,total_books
0,Penguin Books,42


<div style="border:solid gray 2px; padding: 20px">

Издательство Penguin Books выпустило самое большое количество книг (42 книги).

### Определите автора с самой высокой средней оценкой книг  (книги с 50 и более оценками)

In [25]:
author_with_highest_avg_rating = "SELECT author, avg_rating FROM authors INNER JOIN(SELECT books.author_id AS author_id, subquery.avg_rating AS avg_rating FROM books INNER JOIN (SELECT ratings.book_id as book_id, AVG(ratings.rating) AS avg_rating, COUNT(ratings.rating_id) as total_ratings FROM ratings GROUP BY ratings.book_id HAVING COUNT(ratings.rating_id) >= 50) AS subquery ON subquery.book_id = books.book_id) AS subquery2 ON subquery2.author_id = authors.author_id ORDER BY avg_rating DESC"

pd.io.sql.read_sql(author_with_highest_avg_rating, con = engine)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.414634
1,J.R.R. Tolkien,4.391892
2,J.K. Rowling/Mary GrandPré,4.287500
3,Markus Zusak/Cao Xuân Việt Khương,4.264151
4,J.K. Rowling/Mary GrandPré,4.246575
5,Louisa May Alcott,4.192308
6,J.K. Rowling/Mary GrandPré,4.186667
7,J.R.R. Tolkien,4.125000
8,Rick Riordan,4.080645
9,William Golding,3.901408


<div style="border:solid gray 2px; padding: 20px">

Самый высокий рейтинг у авторов: J.K. Rowling/Mary GrandPré. Рейтинг составил - 4.14. Для анализа были выбраны книга с 50 и более оценками.  

### Cреднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [24]:
avg_reviews = "SELECT AVG(subquery2.cnt_review) AS avg_count_rating FROM (SELECT reviews.username, COUNT (DISTINCT reviews.review_id) AS cnt_review FROM reviews RIGHT JOIN(SELECT ratings.username, COUNT(ratings.rating_id) AS cnt_rating FROM ratings GROUP BY ratings.username HAVING COUNT(ratings.rating_id) >= 50) AS subquery ON subquery.username = reviews.username GROUP BY reviews.username) AS subquery2";

pd.io.sql.read_sql(avg_reviews, con = engine)

,avg_count_rating
0,24.222222


<div style="border:solid gray 2px; padding: 20px">
    
В среднем , пользователи, которые проставили книгам более 50 оценок, написали около 24 обзора. 